In [ ]:
!pip install yfinance

In [131]:
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [132]:
start_date = '2021-01-01'
end_date = '2023-01-01'
ticker = 'TSLA'
data = yf.download(ticker, start_date, end_date)
data.tail(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500
2022-12-29,120.389999,123.570000,117.500000,121.820000,121.820000,221923300
2022-12-30,119.949997,124.480003,119.750000,123.180000,123.180000,157777300


In [133]:
def EMA(data, period=20, column="Close"):
  return data[column].ewm(span=period, adjust=False).mean()

In [134]:
def MACD(data, period_short=12, period_long=26, period_signal=9, column="Close"):
  ShortEMA = EMA(data, period_short, column=column)
  LongEMA = EMA(data, period_long, column=column)
  data["Main line"] = ShortEMA - LongEMA
  data["Signal line"] = EMA(data, period_signal, column="Main line")
  return data

In [135]:
def strategy(sig):
  buy = []
  sell = []
  flag = -1

  for i in range(0, len(sig)):
    if sig['Main line'][i] > sig["Signal line"][i]:
      sell.append(np.nan)
      if flag != 1:
        buy.append(sig["Close"][i])
        flag = 1
      else:
        buy.append(np.nan)
    elif sig['Main line'][i] < sig["Signal line"][i]:
      buy.append(np.nan)
      if flag != 0:
        sell.append(sig["Close"][i])
        flag = 0
      else:
        sell.append(np.nan)
    else:
      buy.append(np.nan)
      sell.append(np.nan)
  return (buy, sell)

In [136]:
nobs = 5
train, test = data[:-nobs], data[-nobs:]
print(train.shape, test.shape)

(498, 6) (5, 6)


In [137]:
MACD(train)

,Open,High,Low,Close,Adj Close,Volume,Main line,Signal line
Date,,,,,,,,
2021-01-04,239.820007,248.163330,239.063339,243.256668,243.256668,145914600,0.000000,0.000000
2021-01-05,241.220001,246.946671,239.733337,245.036667,245.036667,96735600,0.141994,0.028399
2021-01-06,252.830002,258.000000,249.699997,251.993332,251.993332,134100000,0.806573,0.184034
2021-01-07,259.209991,272.329987,258.399994,272.013336,272.013336,154496700,2.915100,0.730247
2021-01-08,285.333344,294.829987,279.463318,293.339996,293.339996,225166500,6.235131,1.831224
...,...,...,...,...,...,...,...,...
2022-12-16,159.639999,160.990005,150.039993,150.229996,150.229996,139032200,-11.892506,-10.198989
2022-12-19,154.000000,155.250000,145.820007,149.869995,149.869995,139390600,-12.472495,-10.653690
2022-12-20,146.050003,148.470001,137.660004,137.800003,137.800003,159563300,-13.747615,-11.272475


In [138]:
res = strategy(train)
train["Buy signal"] = res[0]
train["Sell signal"] = res[1]

In [139]:
train["Close_box"], lmbda = stats.boxcox(train.Close)

In [140]:
arima_model = ARIMA(train.Close_box, order=(8, 1, 2)).fit()

In [141]:
def invboxcox(y, lmbda):
    if lmbda == 0:
        return np.exp(y)
    else:
        return np.exp(np.log(lmbda * y + 1) / lmbda)

In [142]:
arima_preds = arima_model.predict(train.shape[0], train.shape[0] + nobs-1)
arima_res = invboxcox(arima_preds, lmbda)

In [144]:
fig = go.Figure(data=[go.Candlestick(x=train.index,
                                    open=train.Open,
                                    high=train.High,
                                    low=train.Low,
                                    close=train.Close,
                                    name='Candlestick chart'),
                      go.Scatter(x=train.index, y=train['Main line'], line=dict(color='LightSkyBlue', width=2), name='Main line'),
                      go.Scatter(x=train.index, y=train['Signal line'], line=dict(color='orange', width=2), name='Signal line'),
                      go.Scatter(x=train.index, y=train['Buy signal'], mode='markers',
                                 marker=dict(symbol='triangle-up', color='green', size=20), name='Buy signal'),
                      go.Scatter(x=train.index, y=train['Sell signal'], mode='markers',
                                 marker=dict(symbol='triangle-down', color='red', size=20), name='Sell signal'),
                      go.Scatter(x=test.index, y=arima_res, line=dict(color='magenta', width=2), name='ARIMA prediction')])
fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title='Сигналы к торговле в зависимости от значения индикатора MACD',
                  yaxis_title='Стоимость акции TSLA',
                  margin_l=0, margin_b=0, margin_r=0, margin_t=50)
fig.show()